# Dynamic Time Warping for Non-Anthropomorphic Hand Data

In [ ]:
pip install fastdtw

In [ ]:
# Set up:
import pandas as pd
import numpy as np

# Plotting Packages
import matplotlib.pyplot as plt

import matplotlib as mpl
from mpl_toolkits import mplot3d

mpl.rcParams['figure.dpi'] = 150
savefig_options = dict(format="png", dpi=150, bbox_inches="tight")

# Computation packages
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [ ]:
def dtw_data_import(set_num, run):

#     Y-position of hand and end-effector tend to be most consistently aligned. Use those to discover the time warp mapping

    filename = "C:\\Users\\jmoln\\Documents\\Projects\\NonAnthroHands_User_Study\\data\\positions\\positions_"+str(set_num)+"_data\\j2s6s300_end_effectorMotion_"+str(run)+".csv"
    # Import data from csvs
    end_eff_raw = pd.read_csv(filename)
    end_eff_data = end_eff_raw.to_numpy()
    
    filename = "C:\\Users\\jmoln\\Documents\\Projects\\NonAnthroHands_User_Study\\data\\positions\\positions_"+str(set_num)+"_data\\RightHandAnchorMotion_"+str(run)+".csv"
    hand_raw = pd.read_csv(filename)
    hand_data = hand_raw.to_numpy()
    

    return end_eff_data, hand_data

In [ ]:
def norm_data(set_num, run, end_eff_data, hand_data):

    x = (end_eff_data[...,0],end_eff_data[...,2])
    y = (hand_data[...,0],hand_data[...,2])
    
    # Normalize x and y to prevent scaling issues from creating DTW misalignment

    scale_x = 1/(np.max(x[1])-np.min(x[1]))
    scale_y = 1/(np.max(y[1])-np.min(y[1]))

    center_x = np.mean(x[1])
    center_y = np.mean(y[1])

    x_norm = (x[0],(x[1] - center_x)*scale_x)
    y_norm = (y[0],(y[1] - center_y)*scale_y)

    # TO-DO: trim ends for cleaner DTW
    # (Not done yet)

    dtw_distance, warp_path = fastdtw(x_norm[1], y_norm[1], dist=euclidean) 
    plot_norm(set_num, run, warp_path, x_norm, y_norm)
    
    return warp_path

In [ ]:
def plot_norm(set_num, run, warp_path, x_norm, y_norm):
    # Show normalized plots
    fig, ax = plt.subplots(figsize=(12, 6))

    # Remove the border and axes ticks
    fig.patch.set_visible(True)
    ax.axis('on')

    max_distance = 0;
    i = 0;

    for [map_x, map_y] in warp_path:

        ax.plot([map_x, map_y], [x_norm[1][map_x], y_norm[1][map_y]], '--k', linewidth=0.2)
        temp_arr2 = np.array((x_norm[1][map_x], y_norm[1][map_y]))
        max_distance = np.maximum(max_distance, np.linalg.norm(temp_arr2))
        i=i+1


    ax.plot(x_norm[1][:], '-ro', label='x', linewidth=0.2, markersize=2, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
    ax.plot(y_norm[1][:], '-bo', label='y', linewidth=0.2, markersize=2, markerfacecolor='skyblue', markeredgecolor='skyblue')

    ax.set_title("Normalized DTW Distance", fontsize=10, fontweight="bold")
    plt.savefig('NormDTW_Y_'+str(run)+'.png')
    plt.close('all')
    
    return

In [ ]:
# Take the warp_path generated from normalized hand/URDF data and use that to align all other hand data
def full_align(warp_path, end_eff_data, hand_data):
    # Time marks: 
    time_URDF = end_eff_data[...,0]
    time_hand = hand_data[...,0]

    # remember that x = end_eff_pos
    #               y = hand_pos

    # Z-data (forward/back) is offset by the distance between the viewer and the robot. Let's remove that distance for comparison purposes

    wp_size = len(warp_path)
    time_URDF_aligned = np.zeros(wp_size)
    time_hand_aligned = np.zeros(wp_size)
    end_eff_pos_aligned = np.zeros((wp_size,3))
    end_eff_rot_aligned = np.zeros((wp_size,3))
    hand_pos_aligned = np.zeros((wp_size,3))
    hand_rot_aligned = np.zeros((wp_size,3))

    for i, [map_x, map_y] in enumerate(warp_path, start=0):   
        time_URDF_aligned[i] = time_URDF[map_x]
        time_hand_aligned[i] = time_hand[map_y]
        end_eff_pos_aligned[i][0:3] = end_eff_data[map_x][1:4]
        end_eff_rot_aligned[i][0:3] = end_eff_data[map_x][4:]
        hand_pos_aligned[i][0:3]    = hand_data[map_y][1:4]
        hand_rot_aligned[i][0:3]    = hand_data[map_y][4:]
    
    return time_URDF_aligned, time_hand_aligned, end_eff_pos_aligned, end_eff_rot_aligned, hand_pos_aligned, hand_rot_aligned

In [ ]:
def plot_pos(set_num, run, warp_path, end_eff_pos_aligned, hand_pos_aligned):
    fig, ax = plt.subplots(figsize=(15, 10))

    # Show the border and axes ticks,
    fig.patch.set_visible(True)
    ax.axis('on')

    ax = plt.axes(projection='3d')

    # ax.plot(time_URDF_aligned, end_eff_pos_aligned[:].T[0], '-ro', label='End-effector position', \
    #     linewidth=0.2, markersize=2, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
    # ax.plot(time_URDF_aligned, end_eff_pos_aligned[:].T[1], '-ro', label='End-effector position', \
    #     linewidth=0.2, markersize=2, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
    # ax.plot(time_URDF_aligned, end_eff_pos_aligned[:].T[2], '-ro', label='End-effector position', \
    #     linewidth=0.2, markersize=2, markerfacecolor='lightcoral', markeredgecolor='lightcoral')

    # Unity uses a left-handed coordinate system, so plot your position data in the orientation in which it was gathered:
    #  X moving left to right, Z moving front to back, and Y pointing up and down
    ax.scatter(end_eff_pos_aligned[:].T[0], -end_eff_pos_aligned[:].T[2], end_eff_pos_aligned[:].T[1], \
                c=time_URDF_aligned/max(time_URDF_aligned), cmap='Reds', label='End-effector position')
    ax.scatter(hand_pos_aligned[:].T[0]   , -hand_pos_aligned[:].T[2]   , hand_pos_aligned[:].T[1]   , \
               c=time_hand_aligned/max(time_hand_aligned), cmap='Blues', label='Hand position')

    # ax.plot3D(end_eff_pos_aligned[:].T[0], end_eff_pos_aligned[:].T[1], end_eff_pos_aligned[:].T[2], \
    #     '-ro', label='End-effector position', linewidth=0.2, markersize=2, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
    # ax.plot3D(hand_pos_aligned[:].T[0]   , hand_pos_aligned[:].T[1]   , hand_pos_aligned[:].T[2]   , \
    #     '-bo', label='Hand position', linewidth=0.2, markersize=2, markerfacecolor='skyblue', markeredgecolor='skyblue')

    for [map_x, map_y] in warp_path: 
        ax.plot3D([end_eff_pos_aligned[map_x].T[0], hand_pos_aligned[map_y].T[0]], \
                  [-end_eff_pos_aligned[map_x].T[2], -hand_pos_aligned[map_y].T[2]],\
                  [end_eff_pos_aligned[map_x].T[1], hand_pos_aligned[map_y].T[1]], \
                  '--k', linewidth=0.2)

    ax.set_xlabel('Horizontal position')
    ax.set_ylabel('Forward/Back position')
    ax.set_zlabel('Vertical position')

    ax.set_title("DTW Alignment of Hand and URDF End-Effector Position", fontsize=10, fontweight="bold")
    plt.savefig('DTW_Pos'+str(run)+'.png')
    plt.close('all')
    
    return

In [ ]:
def clean_rot_data(set_num, run, hand_rot_aligned):
    # Fix angle inversion issues for hand data

    fig, ax = plt.subplots(figsize=(10, 7))

    for i, [x_rot,y_rot,z_rot] in enumerate(hand_rot_aligned, start=1):
        # Singularities should occur in all axes simultaneously
        if i==len(hand_rot_aligned):
            continue
        elif np.abs(hand_rot_aligned[i].T[0] - hand_rot_aligned[i-1].T[0])>np.abs(hand_rot_aligned[i].T[0] + hand_rot_aligned[i-1].T[0]):
    #         print(time_hand_aligned[i], hand_rot_aligned[i-1], hand_rot_aligned[i])
            hand_rot_aligned[i] = -hand_rot_aligned[i]
        elif np.abs(hand_rot_aligned[i].T[1] - hand_rot_aligned[i-1].T[1])>np.abs(hand_rot_aligned[i].T[1] + hand_rot_aligned[i-1].T[1]):
    #         print(time_hand_aligned[i], hand_rot_aligned[i-1], hand_rot_aligned[i])
            hand_rot_aligned[i] = -hand_rot_aligned[i]
        elif np.abs(hand_rot_aligned[i].T[2] - hand_rot_aligned[i-1].T[2])>np.abs(hand_rot_aligned[i].T[2] + hand_rot_aligned[i-1].T[2]):
    #         print(time_hand_aligned[i], hand_rot_aligned[i-1], hand_rot_aligned[i])
            hand_rot_aligned[i] = -hand_rot_aligned[i]


    # for i, [x_rot,y_rot,z_rot] in enumerate(hand_rot_aligned, start=2):
    #     # Singularities should occur in all axes simultaneously
    #     if np.abs(hand_rot_aligned[i].T[0] - hand_rot_aligned[i-2].T[0])>np.abs(hand_rot_aligned[i].T[0] + hand_rot_aligned[i-2].T[0]):
    #         print(time_hand_aligned[i], hand_rot_aligned[i-2], hand_rot_aligned[i])
    #         hand_rot_aligned[i] = -hand_rot_aligned[i]
    #     elif np.abs(hand_rot_aligned[i].T[1] - hand_rot_aligned[i-2].T[1])>np.abs(hand_rot_aligned[i].T[1] + hand_rot_aligned[i-2].T[1]):
    #         print(time_hand_aligned[i], hand_rot_aligned[i-2], hand_rot_aligned[i])
    #         hand_rot_aligned[i] = -hand_rot_aligned[i]
    #     elif np.abs(hand_rot_aligned[i].T[2] - hand_rot_aligned[i-2].T[2])>np.abs(hand_rot_aligned[i].T[2] + hand_rot_aligned[i-2].T[2]):
    #         print(time_hand_aligned[i], hand_rot_aligned[i-2], hand_rot_aligned[i])
    #         hand_rot_aligned[i] = -hand_rot_aligned[i]

    ax.plot(time_hand_aligned, hand_rot_aligned[:].T[0], '-ko', label='x', linewidth=0.2, markersize=2, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
    ax.plot(time_hand_aligned, hand_rot_aligned[:].T[1], '-bo', label='x', linewidth=0.2, markersize=2, markerfacecolor='skyblue', markeredgecolor='skyblue')
    ax.plot(time_hand_aligned, hand_rot_aligned[:].T[2], '-ro', label='x', linewidth=0.2, markersize=2, markerfacecolor='red', markeredgecolor='red')

    plt.savefig('DTW_Rot_corrected_'+str(run)+'.png')
    plt.close('all')
    
    return hand_rot_aligned

In [ ]:
def plot_rot(set_num, run, warp_path, end_eff_rot_aligned, hand_rot_aligned):
    # Plot DTW-aligned hand/end-effector orientation

    fig, ax = plt.subplots(figsize=(15, 10))

    # Show the border and axes ticks
    fig.patch.set_visible(True)
    ax.axis('on')

    ax = plt.axes(projection='3d')

    ax.scatter(end_eff_rot_aligned[:].T[0], -end_eff_rot_aligned[:].T[2], end_eff_rot_aligned[:].T[1], \
              c=time_URDF_aligned/max(time_URDF_aligned), cmap='Reds', label='End-effector orientation')
    ax.scatter(hand_rot_aligned[:].T[0]   , -hand_rot_aligned[:].T[2]   , hand_rot_aligned[:].T[1]   , \
              c=time_hand_aligned/max(time_hand_aligned), cmap='Blues', label='Hand orientation')

    for [map_x, map_y] in warp_path: 
        ax.plot3D([end_eff_rot_aligned[map_x].T[0], hand_rot_aligned[map_y].T[0]], \
                  [-end_eff_rot_aligned[map_x].T[2], -hand_rot_aligned[map_y].T[2]],\
                  [end_eff_rot_aligned[map_x].T[1], hand_rot_aligned[map_y].T[1]], \
                  '--k', linewidth=0.2)

    ax.set_title("DTW Alignment of Hand and URDF End-Effector Orientation", fontsize=10, fontweight="bold")
    plt.savefig('DTW_Rot'+str(run)+'.png')
    plt.close('all')
    
    return

In [ ]:
# Functions are all defined. Let's grab some data sets and get them ordered
for set_num in range(1):
#     print(set_num)
    for run in range(1,11):
#         print(set_num, run)
        end_eff_data, hand_data = dtw_data_import(set_num, run)
        warp_path = norm_data(set_num, run, end_eff_data, hand_data)
        time_URDF_aligned, time_hand_aligned, end_eff_pos_aligned, end_eff_rot_aligned, hand_pos_aligned, hand_rot_aligned = \
            full_align(warp_path, end_eff_data, hand_data)
        hand_rot_aligned = clean_rot_data(set_num, run, hand_rot_aligned)
        hand_rot_aligned = clean_rot_data(set_num, run, hand_rot_aligned) #Seems to work better if you do it twice for some reason
        plot_pos(set_num, run, warp_path, end_eff_pos_aligned, hand_pos_aligned)
        plot_rot(set_num, run, warp_path, end_eff_rot_aligned, hand_rot_aligned)
        
        np.savez('data_'+str(set_num)+'_'+str(run),time_URDF_aligned=time_URDF_aligned, time_hand_aligned=time_hand_aligned,\
                end_eff_pos_aligned=end_eff_pos_aligned, end_eff_rot_aligned=end_eff_rot_aligned, \
                hand_pos_aligned=hand_pos_aligned, hand_rot_aligned=hand_rot_aligned, \
                set_num=set_num, run=run, warp_path=warp_path, end_eff_data=end_eff_data, hand_data=hand_data)
